# Toxic Comment Detection
Got a lot of comments on your post and don't know how to filter them? We got you!

With this telegram bot - equipped with AI capabilities - you will be able to know whether these comments are positive or not!

# AI for Sentiment Analysis

In [ ]:
!pip install tensorflow==2.13.0
!pip install tflite-support==0.4.4

In [ ]:
from tflite_support.task import text
import requests

"""
Returns a TFLite NL Classifier using the pretrained model for comment sentiment analysis.
"""
def get_model():
  model_url = "https://storage.googleapis.com/download.tensorflow.org/models/tflite/text_classification/text_classification_v2.tflite"
  r = requests.get(model_url)
  with open('text_classification_v2.tflite','wb') as f:
    f.write(r.content)

  classifier = text.NLClassifier.create_from_file("text_classification_v2.tflite")
  return classifier

"""
Using the given classifier, classify whether the input string has positive or negative sentiment.
This function will return True if positive, else False.
"""
def classify_positive_comment(classifier, input):
  classification_result = classifier.classify(input)
  max_score_category = sorted(classification_result.classifications[0].categories, reverse=True, key=lambda x: x.score)[0]

  if max_score_category.category_name == 'Positive':
    return True
  else:
    return False

In [ ]:
classifier = get_model()
classify_positive_comment(classifier, "This is amazing!")

True

# Creating Telegram Chatbot
First, search BotFather in Telegram, and request for a new bot by typing /newbot.

You will be prompted to give your bot a username, and you will get the bot token.

Here, we will create a telegram chatbot that replies to your every message whether it is positive or not.

In [ ]:
!pip install python-telegram-bot

In [ ]:
from telegram import Update
from telegram.ext import filters, ApplicationBuilder, ContextTypes, MessageHandler
import nest_asyncio

async def reply_with_sentiment(update: Update, context: ContextTypes.DEFAULT_TYPE):
  is_positive = classify_positive_comment(classifier, update.message.text)

  if is_positive:
    reply = "This is a positive comment, keep it up!"
  else:
    reply = "This is not a postive comment, be happy and cheer up!"

  await context.bot.send_message(chat_id=update.effective_chat.id, text=reply)

if __name__ == '__main__':
  application = ApplicationBuilder().token('').build()

  sentiment_handler = MessageHandler(filters.TEXT & (~filters.COMMAND), reply_with_sentiment)
  application.add_handler(sentiment_handler)

  nest_asyncio.apply()
  application.run_polling()

RuntimeError: Cannot close a running event loop